In [1]:
import os
import json
import pandas as pd
from functools import reduce

In [36]:
def unpack_dict(data):
    L = []
    for x in data:
        d = {}
        for k,v in x.items():
            if k == 'values':
                for k2,v2 in v.items():
                    d[k2] = v2
            else:
                d[k] = v    
        L.append(d)
    return L

if __name__ == '__main__':
    path = os.getcwd()
    frames =[]
    for files in os.listdir(path):
        if '.json' in files:
            with open(path + '/' + files) as json_data:
                data = json.load(json_data)
                if 'energy' in files:
                    df = pd.DataFrame(data)
                    frames.append(df)
                else:
                    df = pd.DataFrame(unpack_dict(data))
                    col_names = [col_name for col_name in df.columns if col_name not in ['name', 'units', 'year']]
                    df[col_names] = df[col_names].apply(pd.to_numeric, errors='coerce')
                    new_col_name = f'{files.split("_")[0]}_{df.filter(regex="(deficit$)",axis=1).columns[0]}/surplus'
                    deficit = df.filter(regex='(deficit$)',axis=1).columns[0]
                    surplus = df.filter(regex='(surplus$)+',axis=1).columns[0]
                    df[new_col_name] = df[[deficit, surplus]].sum(axis=1)
                    df.drop([deficit, surplus], axis = 1, inplace=True)
                    df = df.rename(columns = { col : files.split("_")[0] + '_' + col for col in ['net_imports', 'year', 'units', 'net_exports']})
                    frames.append(df)

    df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['name'],
                                                how='outer'), frames)


In [37]:
df_merged

,name,gas_reserves,gas_production,gas_consumption,gas_imports,gas_exports,gas_net_imports,gas_year,gas_units,gas_net_exports,...,coal_reserves,coal_production,coal_consumption,coal_imports,coal_exports,coal_net_imports,coal_year,coal_units,coal_net_exports,coal_yearly_deficit/surplus
0,United Arab Emirates,2.150980e+08,3178738.0,2621819.0,769972.0,273903.0,496069.0,2015,Million Cubic Ft,NaN,...,0.000000e+00,0.0,2454173.0,2726156.0,13779.0,2712376.0,2016,Tons,NaN,-2454173.0
1,Ukraine,3.900000e+07,670985.0,1171928.0,500943.0,0.0,500943.0,2015,Million Cubic Ft,NaN,...,3.789191e+10,39260975.0,59357189.0,17248947.0,2251110.0,14997837.0,2016,Tons,NaN,-20096214.0
2,Kazakhstan,8.500000e+07,1515014.0,480637.0,165981.0,434021.0,NaN,2015,Million Cubic Ft,268041.0,...,2.822465e+10,113619501.0,86633850.0,391.0,4273196.0,NaN,2016,Tons,4272805.0,26985651.0
3,Singapore,0.000000e+00,0.0,428018.0,436847.0,8829.0,428018.0,2015,Million Cubic Ft,NaN,...,0.000000e+00,0.0,761679.0,761747.0,68.0,761679.0,2016,Tons,NaN,-761679.0
4,Egypt,7.720000e+07,1742018.0,1692930.0,127487.0,0.0,127487.0,2015,Million Cubic Ft,NaN,...,1.763696e+07,0.0,769412.0,3337265.0,95121.0,3242144.0,2016,Tons,NaN,-769412.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,India,5.039800e+07,1133364.0,1753143.0,659331.0,9535.0,649796.0,2015,Million Cubic Ft,NaN,...,1.077266e+11,761662038.0,966288693.0,215789801.0,1018312.0,214771489.0,2016,Tons,NaN,-204626654.0
204,Russia,1.688228e+09,22728734.0,15538247.0,557977.0,6550933.0,NaN,2015,Million Cubic Ft,5992956.0,...,1.767708e+11,423095348.0,230392143.0,24331105.0,181506237.0,NaN,2016,Tons,157175131.0,192703205.0
205,United States,3.687040e+08,32914647.0,27243858.0,2718094.0,1783512.0,934582.0,2015,Million Cubic Ft,NaN,...,2.541970e+11,728364498.0,731071000.0,9849735.0,60271017.0,NaN,2016,Tons,50421282.0,-2706502.0
206,Madagascar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000e+00,0.0,566337.0,552763.0,0.0,552763.0,2016,Tons,NaN,-566337.0


In [19]:
df_merged.to_csv('worldometers_energy.csv', index=False)